In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# :

In [35]:
import morph_kgc

config = '''
[DataSourceCSV]
mappings=../src/rdf/statute-mapping.ttl

[CONFIGURATION]
output_file=../data/analysis/knowledge-graph/statute-events-rdf.nt
'''

rdf_graph = morph_kgc.materialize(config)

INFO | 2023-07-14 11:40:30,275 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2023-07-14 11:40:32,760 | 80 mapping rules retrieved.
INFO | 2023-07-14 11:40:32,818 | Mapping partition with 1 groups generated.
INFO | 2023-07-14 11:40:32,819 | Maximum number of rules within mapping group: 80.
INFO | 2023-07-14 11:40:32,820 | Mappings processed in 2.540 seconds.
INFO | 2023-07-14 11:40:33,558 | Number of triples generated in total: 8378.


In [38]:
rdf_graph.serialize('../data/analysis/knowledge-graph/statute-events-rdf.nt', 'nt')

c:\Users\gabya\anaconda3\envs\event-extraction\lib\site-packages\rdflib\plugins\serializers\nt.py:40: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=Nf6cf10bc18844335919dfa81bb1de787 (<class 'rdflib.graph.Graph'>)>

# Query

In [47]:
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery

In [45]:
g = Graph()
g.parse('../data/analysis/knowledge-graph/statute-events-rdf.ttl', format='turtle')

<Graph identifier=N6d4e85bca5774d3cb429149af406ca94 (<class 'rdflib.graph.Graph'>)>

¿De qué maneras puede un empleado pagar el salario de sus empleados?

*What are the ways that an employer has to pay to his workers?*

In [83]:
from rdflib.namespace import RDFS

namespaces = {
    'ns1': 'http://semanticweb.cs.vu.nl/2009/11/sem/',
    'ns2': 'http://www.ittig.cnr.it/ontologies/def/ProvisionModel#',
    'rdfs': RDFS
}

worker_rights_query = prepareQuery(
'''
    SELECT ?rightAction ?rightComplement
    WHERE {{
            ?subject a ns1:Actor .
            ?subject rdfs:label "empresario" .
            ?rightRelation a ns2:Right .
            ?rightRelation ns2:hasRightBearer ?subject .
            ?rightRelation ns2:hasRightCounterpart ?object .
            ?object rdfs:label "salario" .
            ?rightRelation ns2:hasRightAction ?action .
            ?action rdfs:label ?rightAction .
            ?rightRelation ns2:hasRightObject ?complement .
            ?complement rdfs:label ?rightComplement .
    }}
''', initNs=namespaces)

# Visualize the results
for r in g.query(worker_rights_query):
    print(f'El emplesario {r.rightAction} (el pago del salario) en {r.rightComplement}')

El emplesario podrá efectuarlo (el pago del salario) en moneda de curso legal
El emplesario podrá efectuarlo (el pago del salario) en cheque
El emplesario podrá efectuarlo (el pago del salario) en modalidad de pago similar
El emplesario podrá efectuarlo (el pago del salario) en entidades de crédito
El emplesario podrá efectuarlo (el pago del salario) en informe al comité de empresa
El emplesario podrá efectuarlo (el pago del salario) en delegados de personal


¿Qué deberes tiene el empresario para/con sus trabajadores?

*What duties an employer has towards his workers?*

In [90]:
from rdflib.namespace import RDFS

namespaces = {
    'ns1': 'http://semanticweb.cs.vu.nl/2009/11/sem/',
    'ns2': 'http://www.ittig.cnr.it/ontologies/def/ProvisionModel#',
    'rdfs': RDFS
}

employer_duty_worker_query = prepareQuery(
'''
    SELECT ?dutyAction ?dutyComplement
    WHERE {{
            ?subject a ns1:Actor .
            ?subject rdfs:label "empresario" .
            ?dutyRelation a ns2:Duty .
            ?dutyRelation ns2:hasDutyBearer ?subject .
            ?dutyRelation ns2:hasDutyCounterpart ?object .
            ?object rdfs:label "trabajador" .
            ?dutyRelation ns2:hasDutyAction ?action .
            ?action rdfs:label ?dutyAction .
            ?dutyRelation ns2:hasDutyObject ?complement .
            ?complement rdfs:label ?dutyComplement .
    }}
''', initNs=namespaces)

# Visualize the results
for r in g.query(employer_duty_worker_query):
    print(f'El empleado {r.dutyAction} {r.dutyComplement} a trabajadores.')

El empleado ofrecer horas complementarias a trabajadores.
El empleado pondrá a disposición salarios devengados en los días intermedios a trabajadores.
